In [2]:
# get the data through the API
import pandas as pd
import requests

# takes url of the dataset, returns a Pandas DataFrame of the dataset
def getNYCdata(url):
    key = 'SECRET' 
    response = requests.get(url, headers={'X-App-Token': key})
    data = response.text
    df = pd.read_json(data)
    return df


# Daily Admissions
url = "https://data.cityofnewyork.us/resource/6teu-xtgp.json?$limit=500000"
df = getNYCdata(url)

In [3]:
df

,inmateid,admitted_dt,discharged_dt,race,gender,inmate_status_code,top_charge
0,10008773,2018-08-23T10:15:28.000,2018-08-23T11:14:00.000,UNKNOWN,M,DE,VTL 1192.3
1,20037818,2018-08-05T19:13:46.000,2018-08-10T16:46:05.000,UNKNOWN,M,DE,NaN
2,20197484,2018-08-14T02:08:36.000,NaN,BLACK,F,DE,120.05
3,20150289,2018-08-02T00:05:22.000,NaN,UNKNOWN,M,DPV,NaN
4,20122449,2018-08-23T18:34:24.000,NaN,UNKNOWN,M,DE,120.05
...,...,...,...,...,...,...,...
354817,20217624,2022-02-28T17:50:59.000,NaN,UNKNOWN,M,DE,265.03
354818,20217625,2022-02-28T18:00:12.000,NaN,UNKNOWN,F,DE,215.50
354819,20217626,2022-02-28T19:18:00.000,NaN,UNKNOWN,M,DE,110-160.15
354820,20217627,2022-02-28T20:57:45.000,NaN,BLACK,M,DE,110-125.25


In [25]:
df['admitted_dt'] = pd.to_datetime(df['admitted_dt'])

In [30]:
df[['admitted_dt']].describe()

<ipython-input-30-1014faecbdea>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df[['admitted_dt']].describe()


,admitted_dt
count,354822
unique,344153
top,2016-05-22 12:32:00
freq,7
first,2014-01-01 07:30:00
last,2022-02-28 22:27:44


In [11]:
df.columns

Index(['inmateid', 'admitted_dt', 'discharged_dt', 'race', 'gender',
       'inmate_status_code', 'top_charge'],
      dtype='object')

In [8]:
df['inmateid'].unique

<bound method Series.unique of 0         10008773
1         20037818
2         20197484
3         20150289
4         20122449
            ...   
354817    20217624
354818    20217625
354819    20217626
354820    20217627
354821    20217628
Name: inmateid, Length: 354822, dtype: int64>

In [256]:
df['admitted_dt'] = pd.to_datetime(df['admitted_dt'])

see if I can cross reference with the codelawmanual sheet to put the charges in plain English
from here: https://www.criminaljustice.ny.gov/crimnet/ccman/ccman.htm
110- prefix doesn't seem to mean anything - but probably does and I can't figure it out
conclusion: add a plaintext version if it's available, do some data cleaning to be more likely to get it

In [257]:
race_dict = {"B" : 'Black', "W" : 'White', "A": 'Asian', "I" : 'American-Indian', "O": 'Other', 'U': 'Unknown'}
df=df.replace({"race": race_dict})

In [258]:
set(df['race'].values)
df.race.fillna('Unknown')

0         UNKNOWN
1         UNKNOWN
2           BLACK
3         UNKNOWN
4         UNKNOWN
           ...   
354817    UNKNOWN
354818    UNKNOWN
354819    UNKNOWN
354820      BLACK
354821      BLACK
Name: race, Length: 354822, dtype: object

In [259]:
gender_dict = {'M': 'Male', 'F': 'Female'}
df=df.replace({"gender": gender_dict})
df['gender']=df.gender.fillna('Unknown')

In [260]:
status_dict = {'CS': 'City Sentenced', 'CSP':'City Sentenced, with VP Warrant',
                'DE': 'Detainee', 'DEP':'Detainee, with Open Case & VP Warrant',
                'DNS': 'Detainee, Newly Sentenced to State Time',
                'DPV': 'Detainee, Technical Parole Violator',
                'SCO': 'State Prisoner, Court Order', 'SSR':'State Ready'}
df=df.replace({"inmate_status_code": status_dict})

In [261]:
# now see what I can do with the codes
charge_codes = pd.read_csv('codedlawmanual.csv', header=0)
charge_codes = charge_codes[['Title','Section','Sub','Class','Short Description', 'Full Description']]

follow format of ###.## like 125.15
if there's a dash or space before, get rid of everything before and including the dash
remove formatting like - or . or " "
if there's a letter after
treat all strings like

In [262]:
import re
import numpy as np

def parse_code(charge):
    charge = str(charge)
    if len(charge)==0:
        return "no match"
    else:
        pattern = '(\d{3}|\d{4})\.(\d{2}|\d{1})'
        code_search = re.search(pattern, charge, re.IGNORECASE)

        if code_search:
            return code_search.group(0)
        else:
            return "no match"

df['parsed_charge'] = df.top_charge.apply(parse_code)
df['parsed_charge']

0           1192.3
1         no match
2           120.05
3         no match
4           120.05
            ...   
354817      265.03
354818      215.50
354819      160.15
354820      125.25
354821      900.00
Name: parsed_charge, Length: 354822, dtype: object

In [263]:
# inputs charge, returns Short Description and Full Description
# description_type = Short Description, Full Description
def cross_reference(charge, description_type):
    # get the first one that matches
    if charge == 'no match':
        return 
    else:
        matches = charge_codes[charge_codes['Section']==charge]
        if not matches.empty:
            row = matches.iloc[0]
            if len(matches)>0:
                row_na = matches[matches['Sub'].isna()]
                if len(row_na)>0:
                    return row_na.iloc[0][description_type]
                else:
                    return row.iloc[1][description_type]
            else:
                return
        else:
            return

In [264]:
arg1 = 4 #"Short Description" 
arg2 = 5 #"Full Description"
df['Charge_Short_Description'] = df.parsed_charge.apply(cross_reference, description_type=arg1)
df['Charge_Full_Description'] = df.parsed_charge.apply(cross_reference, description_type=arg2)

In [265]:
set(df.Charge_Short_Description) # log how many although just use regular charge one for that

{'2',
 '3',
 'ABSCONDING-1',
 'AG CRIM CONTMPT',
 'AG VEH HOMICIDE',
 'AGG CRIM WEAPON',
 'AGG FAMILY OFF',
 'AGG HARASS EMP',
 'AGG HARASS-1',
 'AGG INS FRAUD 4',
 'AGG INTER 1ST',
 'AGG MANSLAUTR-2',
 'AGG SEX ABUSE 2',
 'AGG SEX ABUSE 3',
 'AGGR ASSAULT',
 'ARSON 5',
 'ARSON-2',
 'ARSON-4',
 'ASLT CHILD',
 'ASSAULT',
 'ASSAULT CHILD',
 'ASSAULT-2ND',
 'AUTO STRIPPING1',
 'AUTO STRIPPING3',
 'BAIL JUMPING-1',
 'BAIL JUMPING-2',
 'BAIL JUMPING-3',
 'BOMB-HAZ SUBST1',
 'BOMB-HAZ SUBSTN',
 'BRIB PUB SERV 2',
 'BRIBERY-LAB OFF',
 'BURGLARY-2',
 'CHLD SEX PERFRM',
 'COERCION-1',
 'COMM BRIBERY-1',
 'COMPEL PROST',
 'CONSPIRACY 1',
 'CONSPIRACY 2',
 'CONSPIRACY 3',
 'CONSPIRACY-5',
 'COURSE SEX COND',
 'CPCS-1ST',
 'CPCS-2ND',
 'CPCS-3',
 'CPCS-4',
 'CPCS-5',
 'CPCS-7',
 'CPSP 1',
 'CPSP 2',
 'CPSP 4',
 'CPW-2ND DEGREE',
 'CRIM ANARCHY',
 'CRIM CONTEMPT-1',
 'CRIM DIV PRES-3',
 'CRIM IMPERS-1ST',
 'CRIM IMPERS-2ND',
 'CRIM INTER 1ST',
 'CRIM MISCHIEF-3',
 'CRIM NUISANCE 1',
 'CRIM NUISANCE

In [266]:
# Make categories
# this is experimental bc I don't know the domain well enough to assign categories accurately
def categorize(short_charge):
    vehicle_related = [
                     'VEH ASSAULT-1ST',
                     'VEH ASSAULT-2ND',
                     'VEH MANSLAUTR-1',
                     'VEH MANSLAUTR-2',
                     'UNAUT USE VEH-1',
                     'UNAUT USE VEH-2',
                     'UNAUT USE VEH-3',
                     'AG VEH HOMICIDE']
    
    unknown = ['Unknown']
    
    sex_related = [
                     'CHLD SEX PERFRM',
                     'COMPEL PROST', 
                     'COURSE SEX COND',
                     'SEX TRAFFICKING',
                     'SEXUAL ABUSE 3',
                     'SEXUAL ABUSE-1',
                     'EXPOSURE PERSON',
                     'RAPE 3RD',
                     'RAPE-1',
                     'PRD SX ASLT CHD',
                     'PRED SEX ASLT',
                     'AGG SEX ABUSE 2',
                     'AGG SEX ABUSE 3',
                     'PATRON PROST -1',
                     'PATRON PROST-2',
                     'PATRON PROST-3',
                     'PERS SEX ABUSE',
                     'CRIM SEX ACT-2',
                     'INCEST 1ST DEG',
                     'INDEC MAT-MINOR',
                ]    
    theft = [
             'GRAND LARCENY',
             'GRAND LARCENY 3',
             'PETIT LARCENY',
             'ROBBERY-1',
             'ROBBERY-3',
             'BURGLARY-2',
             'COMM BRIBERY-1',]
    
    drug_related = [
                     'CSCS PRESCRIPN',
                     'CSCS-1',
                     'CSCS-2ND',
                     'CSCS-3',
                     'CSCS-4',
                     'CSCS-5',
                     'CSCS-SCHOOL',
                     'DRUG TRAFFICKER',
                     'CPCS-1ST',
                     'CPCS-2ND',
                     'CPCS-3',
                     'CPCS-4',
                     'CPCS-5',
                     'CPCS-7',
                     'CPSP 1',
                     'CPSP 2',
                     'CPSP 4',]
    
    conspiracy = [
                 'CONSPIRACY 1',
                 'CONSPIRACY 2',
                 'CONSPIRACY 3',
                 'CONSPIRACY-5',]
    
    bail_jumping = [
                 'BAIL JUMPING-1',
                 'BAIL JUMPING-2',
                 'BAIL JUMPING-3',]
    
    assault_harassment = [
             'AG CRIM CONTMPT',
             'AGG CRIM WEAPON',
             'AGG FAMILY OFF',
             'AGG HARASS EMP',
             'AGG HARASS-1',
             'AGG INS FRAUD 4',
             'AGG MANSLAUTR-2',
             'AGGR ASSAULT',
             'STRANGULATION 1',
             'STRANGULATION 2',
             'MANSLAUGHTER-2',
             'MURDER-1ST DEG',
             'MURDER-2',

             'ASLT CHILD',
             'ASSAULT',
             'ASSAULT CHILD',
             'ASSAULT-2ND',
             'GANG ASLT 1ST',
             'GANG ASLT 2ND',

             'HARASSMENT 1ST',
             'HARASSMENT 2ND',

             'STALKING 2ND',
             'STALKING 4TH',        
            ]
    
    menacing = [
                 'MENACE OFFICER',
                 'MENACING 3RD',
                 'MENACING-1ST',
                 'MENACING-2ND',]
    
    firearm_related = [
                 'CRM SAL F/ARM',
                 'CRM SAL FRARM 3',
                 'CRM USE F/ARM-1',
                 'CRM USE F/ARM-2',
                 'SALE FIREARM-1',
                 'SALE FIREARM-2',
                 'CRIM POS WEAP-3',
                 'CRIM POS WEAP-4',
                 'CPW-2ND DEGREE',

                ]
    arson = [
             'ARSON 5',
             'ARSON-2',
             'ARSON-4',]
    
    if short_charge in arson:
        return 'arson'
    if short_charge in firearm_related:
        return 'firearm_related'
    if short_charge in menacing:
        return 'menacing'
    if short_charge in assault_harassment:
        return 'assault_harrassment'
    if short_charge in bail_jumping:
        return 'bail_jumping'
    if short_charge in conspiracy:
        return 'conspiracy'
    if short_charge in drug_related:
        return 'drug_related'
    if short_charge in theft:
        return 'theft'
    if short_charge in sex_related:
        return 'sex_related'
    if short_charge in vehicle_related:
        return 'vehicle_related'
    if short_charge in unknown:
        return 'unknown'
    else:
        return "other"
    
    return 
    

df['Charge Category'] = df.Charge_Short_Description.apply(lambda x: categorize(x))

#df_filtered = df[df['Col1'].isin(allowed_values)]

In [267]:
df.columns

Index(['inmateid', 'admitted_dt', 'discharged_dt', 'race', 'gender',
       'inmate_status_code', 'top_charge', 'parsed_charge',
       'Charge_Short_Description', 'Charge_Full_Description',
       'Charge Category'],
      dtype='object')

In [268]:
df[['race', 'gender','inmate_status_code', 
    'top_charge', 'parsed_charge',
    'Charge_Short_Description',]] = df[['race', 'gender','inmate_status_code', 'top_charge', 
                                        'parsed_charge','Charge_Short_Description',]].fillna('Unknown')

In [269]:
el = []
for charge in [
 'COERCION-1',
]:
    fcharge = df[df.Charge_Short_Description == charge]['Charge_Full_Description'].iloc[0]
    el.append((charge, fcharge))
el

[('COERCION-1', 'Coercion-1st Degree')]

In [270]:
# time analysis
df['admitted_wkday'] = df['admitted_dt'].dt.day_of_week
df['admitted_month'] = df['admitted_dt'].dt.month
df['admitted_year'] = df['admitted_dt'].dt.year
df['admitted_date'] = df['admitted_dt'].dt.date
df['admitted_hour'] = df['admitted_dt'].dt.hour

In [271]:
# we have discharge dates for most of them actually: 284441
# so I could likely graph that

In [272]:
def convert_to_hours(delta):
    if delta is pd.NaT:
        return
    else:
        delta = delta.components
        return delta.days*24 + delta.hours + delta.minutes/60 + delta.seconds/3600

In [273]:
df['discharged_dt'] = pd.to_datetime(df['discharged_dt'])
df['time_held'] = df['discharged_dt']-df['admitted_dt']
df.head()

,inmateid,admitted_dt,discharged_dt,race,gender,inmate_status_code,top_charge,parsed_charge,Charge_Short_Description,Charge_Full_Description,Charge Category,admitted_wkday,admitted_month,admitted_year,admitted_date,admitted_hour,time_held
0,10008773,2018-08-23 10:15:28,2018-08-23 11:14:00,UNKNOWN,M,Detainee,VTL 1192.3,1192.3,Unknown,None,other,3,8,2018,2018-08-23,10,0 days 00:58:32
1,20037818,2018-08-05 19:13:46,2018-08-10 16:46:05,UNKNOWN,M,Detainee,Unknown,no match,Unknown,None,other,6,8,2018,2018-08-05,19,4 days 21:32:19
2,20197484,2018-08-14 02:08:36,NaT,BLACK,F,Detainee,120.05,120.05,ASSAULT-2ND,Assault -2nd Degree,assault_harrassment,1,8,2018,2018-08-14,2,NaT
3,20150289,2018-08-02 00:05:22,NaT,UNKNOWN,M,"Detainee, Technical Parole Violator",Unknown,no match,Unknown,None,other,3,8,2018,2018-08-02,0,NaT
4,20122449,2018-08-23 18:34:24,NaT,UNKNOWN,M,Detainee,120.05,120.05,ASSAULT-2ND,Assault -2nd Degree,assault_harrassment,3,8,2018,2018-08-23,18,NaT


In [274]:
df.time_held[0].components

Components(days=0, hours=0, minutes=58, seconds=32, milliseconds=0, microseconds=0, nanoseconds=0)

In [275]:
df['time_held_hours'] = df.time_held.apply(lambda x: convert_to_hours(x)) 

In [276]:
time_df = df[df['time_held_hours']>0]
time_df.time_held_hours.describe()

count    282115.000000
mean        955.209742
std        2170.180854
min           0.000278
25%          48.832778
50%         150.500278
75%         695.015972
max       32069.928889
Name: time_held_hours, dtype: float64

In [277]:
time_df.time_held.describe()

count                        282115
mean     39 days 19:12:35.071715435
std      90 days 10:10:51.075855548
min                 0 days 00:00:01
25%                 2 days 00:49:58
50%                 6 days 06:30:01
75%         28 days 23:00:57.500000
max              1336 days 05:55:44
Name: time_held, dtype: object

In [278]:
df.to_csv("full_admissions.csv", index=False)